# Debug adaptive tau leap

First, we set up SIR model with time dependent infectivity

In [1]:
from pygom import SimulateOde, Transition, TransitionType
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import sympy

###################
# ODE specification
###################

# Define SIR model
stateList = ['S', 'I', 'R']
paramList = ['beta', 'gamma', 'N']
transitionList = [Transition(origin='S', destination='I', equation='beta*(1+0.99*cos(2*3.14*t/(10)))*S*I/N', transition_type=TransitionType.T),
                  Transition(origin='I', destination='R', equation='gamma*I', transition_type=TransitionType.T)]

n_pop=1e4  # Total population is fixed

beta=0.4
gamma=1/4

params=[('beta', beta), ('gamma', gamma), ('N', n_pop)]

# Initial conditions
i0=10
x0 = [n_pop-i0, i0, 0]


# Set up pygom object (D_F suffix implies Deterministic_Fixed)
model = SimulateOde(stateList, paramList, transition=transitionList)
model.initial_values = (x0, np.float64(0))  # (initial state conditions, initial timepoint)
model.parameters=params

As part of the class initialisation, the string equations get turned into sympy expressions. For example:

In [59]:
eqn=model._transitionVector[0]
print(eqn)
print(type(eqn))

I*S*beta*(0.99*cos(0.628*t) + 1)/N
<class 'sympy.core.mul.Mul'>


We can see which variables/states are present in this equation:

In [60]:
print(eqn.free_symbols)

{N, t, I, S, beta}


This promisingly includes time, t.
If we want to perform operations with the variables, such as derivatives, we can't define the variable externally

In [67]:
state=sympy.symbols("beta")
print(state)
print(state in eqn.free_symbols)
print(sympy.diff(eqn, state))

beta
False
0


Instead, we have to take them from where they were defined in the class:

In [68]:
state=model._paramDict['beta']
print(state)
print(state in eqn.free_symbols)
print(sympy.diff(eqn, state))

beta
True
I*S*(0.99*cos(0.628*t) + 1)/N


Trouble is, when we pull time from the same dictionary, it is not recognised like the other variables

In [69]:
state=model._paramDict['t']
print(state)
print(state in eqn.free_symbols)
print(sympy.diff(eqn, state))

t
False
0
